In [2]:
import requests
import base64
import numpy as np
from PIL import Image
from io import BytesIO

In [3]:
with open('granny/timber_wolf.png', 'rb') as f:
    input_data = base64.b64encode(f.read()).decode()


# input_image = Image.open('granny/granny_8.jpeg')
# input_data = array_to_base64(np.array(input_image))

def query(input_data):
    response = requests.post('http://granny.advml.com/score', json={'data': input_data})
    return response.json()


query(input_data)['output'][0:5]

[[0.28575703501701355, 'timber wolf'],
 [0.13912102580070496, 'red wolf'],
 [0.1217745989561081, 'coyote'],
 [0.02413186803460121, 'dingo'],
 [0.01603207364678383, 'Eskimo dog']]

In [65]:
def array_to_base64(arr):
    img = Image.fromarray(arr)
    with BytesIO() as buffer:
        # PNG FOR GRANNY, JPEG FOR GRANNY2
        img.save(buffer, 'jpeg')  # must be png to achive same score
        return base64.b64encode(buffer.getvalue()).decode()

In [67]:
def find_granny_prob(preds):
    for i, pred in enumerate(preds):
        if pred[1] == 'Granny Smith':
            return preds[i][0]
    return 0.0


def unnorm(arr):
    arr = arr * 255
    return arr.astype(np.uint8)

In [14]:
query(img_input)['output'][0:5]

0.0006618553888984025

In [50]:

# testing the base64 encoding fuction
img_test = Image.open('granny/timber_wolf.png')

img_arr = np.array(img_test)

img_input = array_to_base64(img_arr)
query(img_input)['output'][0:5] # granny2 compression changes the image to JPEG

[[0.3228543996810913, 'timber wolf'],
 [0.14056767523288727, 'red wolf'],
 [0.12071448564529419, 'coyote'],
 [0.024522099643945694, 'dingo'],
 [0.015388073399662971, 'white wolf']]

In [52]:
import tensorflow as tf
import base64
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

pretrained_model = tf.keras.applications.MobileNetV2(weights='imagenet')
pretrained_model.trainable = False

pretrained_model2 = tf.keras.applications.MobileNet(weights='imagenet')
pretrained_model2.trainable = False

# ImageNet labels
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

decode_predictions2 = tf.keras.applications.mobilenet.decode_predictions

In [71]:


# Helper function to preprocess the image so that it can be inputted in MobileNetV2


# Helper function to extract labels from probability vector
def get_imagenet_label(probs):
    return decode_predictions(probs, top=5)

def get_imagenet_label2(probs):
    return decode_predictions2(probs, top=5)

def get_class_names():
    class_indices = tf.keras.applications.mobilenet_v2.decode_predictions(np.array([list(range(1000))]), top=1000)[0]
    class_indices = np.array(class_indices)
    class_name = list(class_indices[:, 1][::-1])
    class_name = [x.replace('_', ' ') for x in class_name]
    return class_name

imagenet_mean = np.array([127.5, 127.5, 127.5])
imagenet_std = np.array([127.5, 127.5, 127.5])
def preprocess(image, method, colorspace='RGB', pixelnorm='01', k=0) -> tf.Tensor:
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224), method=method)
    
    if colorspace == 'BGR':
        image = image[..., ::-1]
    elif colorspace == 'HSV':
        image = tf.image.rgb_to_hsv(image)
    elif colorspace == 'GRAY':
        image = tf.image.rgb_to_grayscale(image)

    if pixelnorm == '01':
        image = image / 255
    elif pixelnorm == '-11':
        image = (image / 127.5) - 1
    elif pixelnorm == 'mean_std':
        image = (image - imagenet_mean) / imagenet_std
    image = tf.image.rot90(image, k=k)
    # normalize by zscore
    # image = (image / 127.5) - 1
    # image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    image = image[None, ...]
    # print(type(image))
    return image


# with open('granny/timber_wolf.jpg', 'rb') as f:
#     input_data = base64.b64encode(f.read()).decode()
image_raw = tf.io.read_file('granny/granny_8.jpeg')
image = tf.image.decode_image(image_raw)


image_preprocess = preprocess(image, 'bilinear', 'BGR', '01')
image_probs = pretrained_model.predict(image_preprocess)

# get_imagenet_label(image_probs)[0]
l = []
for i in  get_imagenet_label(image_probs)[0]:
    print([i[2], i[1]])
print(l)

1/1 [==============================] - 0s 170ms/step
[0.5381462, 'Granny_Smith']
[0.07272835, 'croquet_ball']
[0.018565644, 'spaghetti_squash']
[0.015848288, 'piggy_bank']
[0.01240812, 'combination_lock']
[]


In [72]:
for method in ['bilinear', 'lanczos3', 'lanczos5', 'bicubic', 'gaussian', 'nearest', 'mitchellcubic']:
    for pixelnorm in ['01', '-11', 'mean_std']:
        for colorspace in ['HSV', 'BGR', 'RGB']:
            for k in range(4):
                image_raw = tf.io.read_file('granny/granny_8.jpeg')
                image = tf.image.decode_image(image_raw)
                image_preprocess = preprocess(image, method=method, pixelnorm=pixelnorm, colorspace=colorspace, k=k)
                # print(image_preprocess.shape)
                image_preprocess = image_preprocess.numpy()
                image_preprocess = image_preprocess.squeeze()
                image_preprocess = unnorm(image_preprocess)
                # print(image_preprocess.shape)
                # print(query(array_to_base64(image_preprocess))['output'][0:5])
                # 
                # image_probs = pretrained_model2.predict(image_preprocess)
                # 
                with open('granny/res3.txt', 'a') as f:
                    f.write(str(query(array_to_base64(image_preprocess))['output'][0:5]))
                    f.write('\n')
                # with open ('granny/results2.txt', 'a') as f:
                #     f.write(str(method) + ' ' + str(pixelnorm) + ' ' + str(colorspace) + ' ' + str(k) + '\n')
                #     for i in  get_imagenet_label2(image_probs)[0]:
                #         f.write(str([i[2], i[1]]) + '\n')
                #     f.write('\n')
                # print(method, pixelnorm, colorspace, k)
                # for i in  get_imagenet_label(image_probs)[0]:
                #     print([i[2], i[1]])

In [ ]:
# 
# plt.figure()
# plt.imshow(image[0] * 0.5 + 0.5)  # To change [-1, 1] to [0,1]
# _, image_class, class_confidence = get_imagenet_label(image_probs)
# plt.title('{} : {:.2f}% Confidence'.format(image_class, class_confidence * 100))
# plt.show()

In [31]:
# get_imagenet_label(image_probs)[0]
l = []
for i in  get_imagenet_label(image_probs)[0]:
    print([i[2], i[1]])
# print(l)

[0.8407195, 'timber_wolf']
[0.032777686, 'red_wolf']
[0.025875296, 'coyote']
[0.017857784, 'white_wolf']
[0.016977008, 'dingo']


In [14]:
_, image_class, class_confidence = get_imagenet_label(image_probs)
print(image_class, class_confidence)

ValueError: not enough values to unpack (expected 3, got 1)

In [17]:
import numpy as np

zero_img = np.zeros((224, 224, 3), dtype=np.uint8)

In [18]:
query(array_to_base64(zero_img))['output'][0:5]

[[0.008500803261995316, 'nematode'],
 [0.005202698055654764, 'matchstick'],
 [0.004718960262835026, 'spotlight'],
 [0.003972220700234175, 'microphone'],
 [0.0036747155245393515, 'hook']]

In [19]:
image = preprocess(zero_img)
image_probs = pretrained_model.predict(image)

1/1 [==============================] - 0s 57ms/step


In [23]:
s = decode_predictions(image_probs)
print(s)

[[('n04286575', 'spotlight', 0.31856725), ('n03729826', 'matchstick', 0.03370584), ('n03196217', 'digital_clock', 0.03311544), ('n04418357', 'theater_curtain', 0.025268512), ('n03637318', 'lampshade', 0.0214586)]]


In [3]:

# try:
# 	raw_input
# except:
# 	raw_input = input

import numpy as np
import time
import os
from PIL import Image

from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

MOBILENET_MEAN = np.array([0.485, 0.456, 0.406])


def orthogonal_perturbation(delta, prev_sample, target_sample):
    """Generate orthogonal perturbation."""
    perturb = np.random.randn(1, 224, 224, 3)
    perturb /= np.linalg.norm(perturb, axis=(1, 2))
    perturb *= delta * np.mean(get_diff(target_sample, prev_sample))
    # Project perturbation onto sphere around target
    diff = (target_sample - prev_sample).astype(np.float32)  # Orthorgonal vector to sphere surface
    diff /= get_diff(target_sample, prev_sample)  # Orthogonal unit vector
    # We project onto the orthogonal then subtract from perturb
    # to get projection onto sphere surface
    perturb -= (np.vdot(perturb, diff) / np.linalg.norm(diff) ** 2) * diff
    # Check overflow and underflow
    overflow = (prev_sample + perturb) - 255 + MOBILENET_MEAN
    perturb -= overflow * (overflow > 0)
    underflow = -MOBILENET_MEAN
    perturb += underflow * (underflow > 0)
    return perturb


def forward_perturbation(epsilon, prev_sample, target_sample):
    """Generate forward perturbation."""
    perturb = (target_sample - prev_sample).astype(np.float32)
    perturb *= epsilon
    return perturb


def get_converted_prediction(sample, classifier):
    """
    The original sample is dtype float32, but is converted
    to uint8 when exported as an image. The loss of precision
    often causes the label of the image to change, particularly
    because we are very close to the boundary of the two classes.
    This function checks for the label of the exported sample
    by simulating the export process.
    """
    sample = (sample + MOBILENET_MEAN).astype(np.uint8).astype(np.float32) - MOBILENET_MEAN
    label = decode_predictions(classifier.predict(sample), top=1)[0][0][1]
    return label


def save_image(sample, classifier, folder):
    """Export image file."""
    label = get_converted_prediction(np.copy(sample), classifier)
    sample = sample[0]
    # Reverse preprocessing, see https://github.com/keras-team/keras/blob/master/keras/applications/imagenet_utils.py
    sample += MOBILENET_MEAN
    sample = sample[..., ::-1].astype(np.uint8)
    # Convert array to image and save
    sample = Image.fromarray(sample)
    id_no = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    # Save with predicted label for image (may not be adversarial due to uint8 conversion)
    sample.save(os.path.join(os.getcwd(), folder, "{}_{}.png".format(id_no, label)))


def preprocess(sample_path):
    """Load and preprocess image file."""
    img = image.load_img(sample_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x


def get_diff(sample_1, sample_2):
    """Channel-wise norm of difference between samples."""
    return np.linalg.norm(sample_1 - sample_2, axis=(1, 2))


def boundary_attack():
    # Load model, images and other parameters
    classifier = MobileNetV2(weights='imagenet')
    initial_sample = preprocess('granny/timber_wolf.png')
    target_sample = preprocess('granny/granny_8.jpeg')
    folder = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    os.mkdir(os.path.join(os.getcwd(), folder))
    save_image(np.copy(initial_sample), classifier, folder)
    attack_class = np.argmax(classifier.predict(initial_sample))
    target_class = np.argmax(classifier.predict(target_sample))

    adversarial_sample = initial_sample
    n_steps = 0
    n_calls = 0
    epsilon = 1.
    delta = 0.1

    # Move first step to the boundary
    while True:
        trial_sample = adversarial_sample + forward_perturbation(epsilon, adversarial_sample, target_sample)
        prediction = classifier.predict(trial_sample)
        n_calls += 1
        if np.argmax(prediction) == attack_class:
            adversarial_sample = trial_sample
            break
        else:
            epsilon *= 0.9

    # Iteratively run attack
    while True:
        print("Step #{}...".format(n_steps))
        # Orthogonal step
        print("\tDelta step...")
        d_step = 0
        while True:
            d_step += 1
            print("\t#{}".format(d_step))
            trial_samples = []
            predictions = []
            for i in np.arange(10):
                trial_sample = adversarial_sample + orthogonal_perturbation(delta, adversarial_sample, target_sample)
                trial_samples.append(trial_sample)
                pred = classifier.predict(trial_sample)
                predictions.append(pred)
            n_calls += 10
            predictions = np.argmax(predictions, axis=1)
            d_score = np.mean(predictions == attack_class)
            if d_score > 0.0:
                if d_score < 0.3:
                    delta *= 0.9
                elif d_score > 0.7:
                    delta /= 0.9
                adversarial_sample = np.array(trial_samples)[np.where(predictions == attack_class)[0][0]]
                break
            else:
                delta *= 0.9
        # Forward step
        print("\tEpsilon step...")
        e_step = 0
        while True:
            e_step += 1
            print("\t#{}".format(e_step))
            trial_sample = adversarial_sample + forward_perturbation(epsilon, adversarial_sample, target_sample)
            prediction = classifier.predict(trial_sample)
            n_calls += 1
            if np.argmax(prediction) == attack_class:
                adversarial_sample = trial_sample
                epsilon /= 0.5
                break
            elif e_step > 500:
                break
            else:
                epsilon *= 0.5

        n_steps += 1
        chkpts = [1, 5, 10, 50, 100, 500]
        if (n_steps in chkpts) or (n_steps % 500 == 0):
            print("{} steps".format(n_steps))
            save_image(np.copy(adversarial_sample), classifier, folder)
        diff = np.mean(get_diff(adversarial_sample, target_sample))
        if diff <= 1e-3 or e_step > 500:
            print("{} steps".format(n_steps))
            print("Mean Squared Error: {}".format(diff))
            save_image(np.copy(adversarial_sample), classifier, folder)
            break

        print("Mean Squared Error: {}".format(diff))
        print("Calls: {}".format(n_calls))
        print("Attack Class: {}".format(attack_class))
        print("Target Class: {}".format(target_class))
        print("Adversarial Class: {}".format(np.argmax(prediction)))



In [4]:
boundary_attack()

1/1 [==============================] - 0s 52ms/step
Step #0...
	Delta step...
	#1
1/1 [==============================] - 0s 56ms/step
	#2
1/1 [==============================] - 0s 54ms/step
	#3
1/1 [==============================] - 0s 54ms/step
	#4
1/1 [==============================] - 0s 52ms/step
	#5
1/1 [==============================] - 0s 53ms/step
	#6
1/1 [==============================] - 0s 51ms/step
	#7
1/1 [==============================] - 0s 60ms/step
	#8
1/1 [==============================] - 0s 54ms/step
	#9
1/1 [==============================] - 0s 58ms/step
	#10
1/1 [==============================] - 0s 54ms/step
	#11
1/1 [==============================] - 0s 53ms/step
	#12
1/1 [==============================] - 0s 58ms/step
	#13
1/1 [==============================] - 0s 56ms/step
	#14
1/1 [==============================] - 0s 52ms/step
	#15
1/1 [==============================] - 0s 53ms/step
	#16
1/1 [==============================] - 0s 61ms/step
	#17
1/1 [=========

KeyboardInterrupt: 